### Coordinate Descent &amp; The Lasso Solution

In [1]:
import pandas as pd
import numpy as np
from numpy.random import rand
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from numpy.linalg import norm

**Generating Random Data for Testing**

- A 100x50 Matrix, X, drawing randomly from a gaussian distribution. 

- A 100x1 Vector, y, with values between 0 and 1 with a random distribution.

In [2]:
#Generating Random Data

np.random.seed(42)

n = 100
p = 50

#X Matrix
X = np.random.standard_normal(size=(n, p))

#y Vector
y = rand(n)

### Traditional Least Squares

**Least Squares Solution**

The following is the derivation and intuition behind the least squares solution for a linear regression, with code to produce the results.

![title](figures/least_squares_derivation.png)

In the above, **n** is the number of observations, **k** is the k<sup>th</sup> observation, and **p** is the number of variables in the given regression.

*The corresponding least square solution code:*

In [3]:
#Least Squares Solution

# INPUTS

#   X: A matrix of variable values from the generated data
#   y: A vector of target values corresponding the X matrix

# OUTPUTS

#   B: A vector of beta coefficients from the least squares solution (Best Linear Estimator)

def least_squares(X_matrix, y_vector):
    
    inverse_term = inv(np.matmul(X_matrix.T,X_matrix))
    xy_product_term = np.matmul(X_matrix.T, y_vector)

    B = np.matmul(inverse_term, xy_product_term)

    return B

**Results**

Getting the best linear estimator beta_hat vector:

In [4]:
#Getting Least Squares Solution for our Random Data

beta_hat_scratch = least_squares(X,y)
print("Best Linear Estimator (Beta-Hat):\n", beta_hat_scratch)

Best Linear Estimator (Beta-Hat):
 [-0.06659337 -0.01104815  0.00991661  0.09283588 -0.05955373 -0.02918583
  0.00997919 -0.04529953  0.0735396   0.10999968 -0.05427661  0.13462753
  0.05801504 -0.11761879  0.19483932  0.05181994 -0.04994695 -0.05230281
  0.08729247  0.08676209  0.12100758  0.04357916 -0.03363947 -0.06115235
 -0.04161027 -0.00957865 -0.05898289  0.10826193  0.07633543  0.03524932
  0.12398377 -0.08308396 -0.11316674 -0.03002763 -0.09739807 -0.08739415
  0.06077032 -0.02386646  0.02168286  0.03622051 -0.07634741  0.01687032
 -0.00052492 -0.04278844  0.09959658 -0.0729741   0.06570058 -0.02300355
  0.09010428  0.01166315]


In [5]:
#SK-Learn Linear Model Solution

# INPUTS

#   X: A matrix of variable values from the generated data
#   y: A vector of target values corresponding the X matrix

# OUTPUTS

#   model: A linear regression model to extract calculated beta coefficients

def linear_model(X_matrix, y_vector):

    regression_model = LinearRegression(fit_intercept=False).fit(X_matrix, y_vector)

    return regression_model


# INPUTS

#   X_matrix: A matrix of variable values from the generated data
#   y_vector: A vector of target values corresponding the X matrix

# OUTPUTS

#   model: A linear regression model to extract calculated beta coefficients

def linear_model_positive(X_matrix, y_vector):

    regression_model = LinearRegression(fit_intercept=False, positive=True).fit(X_matrix, y_vector)

    return regression_model


# INPUTS

#   X_matrix: A matrix of variable values from the generated data
#   y_vector: A vector of target values corresponding the X matrix
#   a: The Lasso alpha value

# OUTPUTS

#   model: A linear regression model to extract calculated beta coefficients

def lasso_model(X_matrix, y_vector, a):

    lasso = Lasso(a, fit_intercept = False)
    lasso.fit(X_matrix, y_vector)

    return lasso


# INPUTS

#   X_matrix: A matrix of variable values from the generated data
#   y_vector: A vector of target values corresponding the X matrix
#   a: The Lasso alpha value

# OUTPUTS

#   model: A linear regression model to extract calculated beta coefficients

def lasso_model_positive(X_matrix, y_vector, a):

    lasso = Lasso(a, fit_intercept = False, positive = True)
    lasso.fit(X_matrix, y_vector)

    return lasso
#def lasso


**Comparing Model Results**

Comparing our optimized betas with that of a SK-learn linear model implementation.

In [6]:
#Getting a Linear Regression Model
model = linear_model(X, y)

In [7]:
model_positive = linear_model_positive(X, y)

In [8]:
lasso = lasso_model(X, y, 1 / n)

In [9]:
lasso_positive = lasso_model_positive(X, y, 1 / n)

In [10]:
#Function Comparison

# INPUTS

#   b_hat_scratch: A vector of coefficient values corresponding the X matrix produced by the least squares solution function
#   b_hat_sklearn: A vector of coefficients values corresponding the X matrix produced by the linear regression model

# OUTPUTS

#   Equality Print Statement

def beta_vector_comparison(b_hat_scratch, b_hat_sklearn):

    #Rounding to avoid conflict
    b_hat_scratch = [np.round(coef, decimals=3) for coef in b_hat_scratch]
    b_hat_sklearn = [np.round(coef, decimals=3) for coef in b_hat_sklearn]

    if(b_hat_scratch == b_hat_sklearn):
        print("Beta Vectors are Equivalent.")
    
    else:
        print("Beta Vectors are Not Equivalent.")


In [11]:
#Beta Stores
beta_hat_sklearn = list(model.coef_)
beta_hat_scratch = list(beta_hat_scratch)
beta_hat_positive_sklearn = list(model_positive.coef_)
beta_hat_lasso_sklearn = list(lasso.coef_)
beta_hat_lasso_positive_sklearn = list(lasso_positive.coef_)

In [12]:
#Conducting an Example Comparison
beta_vector_comparison(beta_hat_scratch, beta_hat_sklearn)

Beta Vectors are Equivalent.


**Solving the partial derivative for B<sub>i</sub>**

![title](figures/least_squares_partial.png)

**Note: We will decompose this further in the following cells.**

In [13]:
#Computing the Partial

# INPUTS

#   b_hat_scratch: A vector of coefficient values corresponding the X matrix produced by the least squares solution function
#   

# OUTPUTS

#   The ith beta corresponding to the partial derivative

def compute_partial_derivative(b_hat_scratch, i):

    return b_hat_scratch[i - 1]

Computing the the beta coefficient for variable x<sub>4</sub>.

In [14]:
print("Beta Coefficient for x_4:",compute_partial_derivative(beta_hat_scratch, 4))

Beta Coefficient for x_4: 0.09283588424222208


### Coordinate Descent

**Coordinate Descent**

First, let us decompose the traditional least squares solution further by verifying the following:

![title](figures/Least_Squares_Update.png)

In [15]:
# INPUTS

#   n: The length of the beta vector

# OUTPUTS

#   betas: A 1xn vector of beta values from the uniform distribution

def initialize_betas(n):

    betas = np.random.uniform(0,1,n)
    return betas

In [16]:
# INPUTS

#   betas: A vector of current beta values
#   y_data: A vector of target values corresponding the X matrix
#   x_data: A matrix of variable values from the generated data
#   i_column: The index of the column to remove from the x_data matrix and optimze the beta

# OUTPUTS

#   beta_update: The new beta value for the i-th element of the beta vector

def update_rule(betas, y_data, x_data, i_column):

    Xi = x_data[:,i_column]
    X_i = np.delete(x_data, i_column, axis = 1)
    betas_i = np.delete(betas, i_column)
   
    numerator_update = np.dot(Xi.T, (y- np.dot(X_i, betas_i)))
    denominator_update = np.dot(Xi.T, Xi)

    beta_update = numerator_update / denominator_update

    return beta_update

In [17]:
# INPUTS

#   current_beta: The current beta value
#   previous_beta: The prior beta value

# OUTPUTS

#   converged_flag: Boolean flag indicating if the current_beta and previous_beta values have converged

def convergence(current_beta, previous_beta):

    converged_flag = True
    for i in range(len(current_beta)):
        if(np.abs(current_beta[i] - previous_beta[i]) > 1e-12):

            converged_flag = False
            print("_Failed to Converge_")
            return converged_flag
    
    if(converged_flag):
        
        print("_Converged_")
        return converged_flag

In [18]:
# INPUTS

#   current_beta: The current beta value
#   previous_beta: The prior beta value

def abs_difference(current_beta, previous_beta):

    length = len(current_beta)
    sums = 0

    for i in range(len(current_beta)):
        sums += np.abs(current_beta[i] - previous_beta[i])
    
    print("_Average Absoltute Beta Difference_", sums/length)

**Solving the Lasso Solution**

The second term of the lasso objective function is non-differentiable; however, with the sub-gradient, we can achieve the following for optimization:

![title](figures/Lasso_Solution.png)


In [19]:
# INPUTS

#   alpha: The Lasso alpha value
#   i_column: The index of the column to remove from the x_data matrix and optimze the beta
#   x_data: A matrix of variable values from the generated data
#   y_data: A vector of target values corresponding the X matrix
#   betas: A vector of current beta values

# OUTPUTS

#   The beta value for Lasso with  thresholding applied

def lasso_solver(alpha, i_column, x_data, y_data, betas):

    Xi = x_data[:,i_column]
    non_differentiable = alpha / norm(Xi)**2

    g = update_rule(betas, y_data, x_data, i_column)

    if(g < - non_differentiable):
        return g + non_differentiable
    
    elif(g > non_differentiable):
        return g - non_differentiable
    
    else:
        return 0

In [20]:
# INPUTS

#   x_data: A matrix of variable values from the generated data
#   y_data: A vector of target values corresponding the X matrix
#   betas: A vector of current beta values
#   lasso_flag: A boolean to indicate whether the function should apply Lasso (true) or least squares (false)
#   positive_flag: A boolean to indicate whether the function should disallow negative beta values in the model (will be floored to zero)
#   alpha: The Lasso alpha value
#   num_iter: The maximum number of iterations to attempt convergence

# OUTPUTS

#   Optimized Betas or Increase Iter message

def coordinate_descent(x_data, y_data, betas, lasso_flag, positive_flag, alpha, num_iter):

    _betas = betas #initialized betas
    previous_betas = list(betas) #initiaized betas that will be used to retreive previous iteration
    print("_Coordinate Descent Optimization_\n", "_Lasso_ -> " + str(lasso_flag), "| _Positive Solution_ -> " + str(positive_flag))
    for i in range(num_iter): #Iterating over our num_iter
        print("========================================================================")
        print("Iteration:", i+1)
        
        for j in range(len(_betas)): #Iterating over the beta vector
            
            if(not lasso_flag and not positive_flag): #Least Squares Solution
                _betas[j] = update_rule(_betas, y_data, x_data, j) #Calling the update rule function
                
            elif(not lasso_flag and positive_flag): #Least Squares Positive Solution
                if(update_rule(_betas, y_data, x_data, j) < 0): #If Negative Update, set it to zero
                    _betas[j] = 0
                else:
                    _betas[j] = update_rule(_betas, y_data, x_data, j) #Non-Negative Update

            elif(lasso_flag and not positive_flag): #Lasso Solution
                _betas[j] = lasso_solver(alpha, j, x_data, y_data, _betas) #Lasso solver for differentiable and sugbradient

            else:
                if(lasso_solver(alpha, j, x_data, y_data, _betas) < 0): #Lasso Positive solver
                    _betas[j] = 0
                else:
                    _betas[j] = lasso_solver(alpha, j, x_data, y_data, _betas) #Non-Negative Update

        convergence_check = convergence(_betas, previous_betas) #Check to see if we converge

        if(convergence_check): #If converged we are finished - print the results and return our optimized betas
            print("Optimized Betas:")
            print(_betas)
            print("\n_Optimization Finished_")
            return _betas
        
        else: #Continue the loop and set the previous betas to our current betas and return to the loop
            abs_difference(_betas, previous_betas)
            previous_betas = list(_betas)
            
    if(not convergence_check): #Let the user know we need to set more iterations for a potential convergence
        print("\nIterations Provided Failed to Converge. Please Increase your Num. Iterations.")

**Algorithm Performance**

Least Squares Coordinate Descent & **Verification** of Least Squares Coordinate Descent.

In [21]:
betas = initialize_betas(p)

In [22]:
optimized_betas = coordinate_descent(X, y, betas, False, False, 0.1, 250)

_Coordinate Descent Optimization_
 _Lasso_ -> False | _Positive Solution_ -> False
Iteration: 1
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.4758045720123338
Iteration: 2
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.1381135958772933
Iteration: 3
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.07011813915461607
Iteration: 4
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.03808849175530265
Iteration: 5
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.023620483529804462
Iteration: 6
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.01564236594627725
Iteration: 7
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.01063961043240246
Iteration: 8
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.007370356226551931
Iteration: 9
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.005163998584864049
Iteration: 10
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.003647220246183102
Iteration: 11


In [23]:
beta_vector_comparison(list(optimized_betas), beta_hat_sklearn)

Beta Vectors are Equivalent.


Positive Least Squares Coordinate Descent

In [24]:
optimized_betas = coordinate_descent(X, y, betas, False, True, 0.1, 250)

_Coordinate Descent Optimization_
 _Lasso_ -> False | _Positive Solution_ -> True
Iteration: 1
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.03390612910416794
Iteration: 2
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.010943571651437192
Iteration: 3
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.00264256865654237
Iteration: 4
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.000470124000621037
Iteration: 5
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.0001570562347171791
Iteration: 6
_Failed to Converge_
_Average Absoltute Beta Difference_ 6.0909199764705993e-05
Iteration: 7
_Failed to Converge_
_Average Absoltute Beta Difference_ 2.4181694989288888e-05
Iteration: 8
_Failed to Converge_
_Average Absoltute Beta Difference_ 1.1212589351477817e-05
Iteration: 9
_Failed to Converge_
_Average Absoltute Beta Difference_ 5.332324303583109e-06
Iteration: 10
_Failed to Converge_
_Average Absoltute Beta Difference_ 2.4610221857281067e-06

In [25]:
beta_vector_comparison(list(optimized_betas), beta_hat_positive_sklearn)

Beta Vectors are Equivalent.


**Algorithm Performance**

Lasso Coordinate Descent & **Verification** of Lasso Coordinate Descent.

In [26]:
optimized_betas = coordinate_descent(X, y, betas, True, False, 1, 250)

_Coordinate Descent Optimization_
 _Lasso_ -> True | _Positive Solution_ -> False
Iteration: 1
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.02016364884099815
Iteration: 2
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.008110159493115988
Iteration: 3
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.004264729082432937
Iteration: 4
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.002110380470049449
Iteration: 5
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.0010641092501063125
Iteration: 6
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.0005564377779341905
Iteration: 7
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.00030645495406017033
Iteration: 8
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.00017696570409378098
Iteration: 9
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.00010064253671815658
Iteration: 10
_Failed to Converge_
_Average Absoltute Beta Difference_ 5.719251872986112e-05

In [27]:
beta_vector_comparison(list(optimized_betas), beta_hat_lasso_sklearn)

Beta Vectors are Equivalent.


Positive Lasso Coordinate Descent

In [28]:
optimized_betas = coordinate_descent(X, y, betas, True, True, 1, 250)

_Coordinate Descent Optimization_
 _Lasso_ -> True | _Positive Solution_ -> True
Iteration: 1
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.022191163748320024
Iteration: 2
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.0066696906832232105
Iteration: 3
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.0016787350931040668
Iteration: 4
_Failed to Converge_
_Average Absoltute Beta Difference_ 0.0003032241450345616
Iteration: 5
_Failed to Converge_
_Average Absoltute Beta Difference_ 6.656538309367143e-05
Iteration: 6
_Failed to Converge_
_Average Absoltute Beta Difference_ 1.9777227493675865e-05
Iteration: 7
_Failed to Converge_
_Average Absoltute Beta Difference_ 5.778080192426101e-06
Iteration: 8
_Failed to Converge_
_Average Absoltute Beta Difference_ 1.826165926051282e-06
Iteration: 9
_Failed to Converge_
_Average Absoltute Beta Difference_ 6.190773765061414e-07
Iteration: 10
_Failed to Converge_
_Average Absoltute Beta Difference_ 2.1743825962042967e-

In [29]:
beta_vector_comparison(list(optimized_betas), beta_hat_lasso_positive_sklearn)

Beta Vectors are Equivalent.


**Conclusion**

All vectors are equivalent for the Lasso and Least Squares solution with respect to our implementation and SK-Learn. Note, we had to round our betas conservatively because there was a case where a select few of the betas were nearly the same but off by a very small margin in the 5th decimal place due to what we believe to be SK-Learn rounding error. Because changing the max iterations and having an extremely strict convergence rule did not change the result for these specific betas, we account this tiny error to rounding issues bewtween our implementation and any rounding that was done in the SK-learn implementation.

With that said, the end result is clear from the output above; we obtained convergence and have ~ beta equivalence with SK-Learn.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=697db241-4591-47f8-8614-1bf3891b369f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>